#Provided Data Exploration
### Adapted from https://github.com/cs109-students/michaeljohns-2015hw/blob/hw5/hw5part1.ipynb

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
## MLJ: Additional Extras
import time
import itertools

In [3]:
import os
# os.environ['PYSPARK_PYTHON'] = '/anaconda/bin/python'

In [4]:
import findspark
findspark.init()
print findspark.find()
# Depending on your setup you might have to change this line of code
#findspark makes sure I dont need the below on homebrew.
#os.environ['SPARK_HOME']="/usr/local/Cellar/apache-spark/1.5.1/libexec/"
#the below actually broke my spark, so I removed it. 
#Depending on how you started the notebook, you might need it.
# os.environ['PYSPARK_SUBMIT_ARGS']="--master local pyspark --executor-memory 4g"

/home/vagrant/spark


In [5]:
import pyspark
conf = (pyspark.SparkConf()
    .setMaster('local[4]')
    .setAppName('pyspark')
    .set("spark.executor.memory", "2g"))
sc = pyspark.SparkContext(conf=conf)

In [6]:
sc._conf.getAll()

[(u'spark.executor.memory', u'2g'),
 (u'spark.master', u'local[4]'),
 (u'spark.rdd.compress', u'True'),
 (u'spark.driver.memory', u'8g'),
 (u'spark.serializer.objectStreamReset', u'100'),
 (u'spark.submit.deployMode', u'client'),
 (u'spark.app.name', u'pyspark')]

In [7]:
import sys
rdd = sc.parallelize(xrange(10),10)
rdd.map(lambda x: sys.version).collect()

['2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 '2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 '2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 '2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 '2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 '2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 '2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 '2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 '2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 

In [8]:
sys.version

'2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]'

In [9]:
from pyspark.sql import SQLContext
sqlsc=SQLContext(sc)

#Load Provided Data

In [10]:
# load the provided lyrics
lyrics_pd_df = pd.read_csv("../../data/provided/all billboard top 100 songs from 1970-2014.csv")  
# lyrics_pd_df = pd.read_csv("../../data/provided/all billboard top 100 songs from 1970-2014.csv", 
#                            names = ['<column 1>','<column 2>']) # if no header

In [11]:
lyrics_pd_df = lyrics_pd_df[['position','year','title.href','title','artist','lyrics']]

In [12]:
lyrics_pd_df.shape

(4500, 6)

In [13]:
lyrics_pd_df.head()

,position,year,title.href,title,artist,lyrics
0,1,1970,https://en.wikipedia.org/wiki/Bridge_over_Trou...,Bridge over Troubled Water,Simon and Garfunkel,When you're weary feeling small When tears are...
1,2,1970,https://en.wikipedia.org/wiki/(They_Long_to_Be...,(They Long to Be) Close to You,The Carpenters,x
2,3,1970,https://en.wikipedia.org/wiki/American_Woman_(...,American Woman,The Guess Who,"American woman, stay away from me American wom..."
3,4,1970,https://en.wikipedia.org/wiki/Raindrops_Keep_F...,Raindrops Keep Fallin' on My Head,B.J. Thomas,Raindrops keep falling on my head Just like th...
4,5,1970,https://en.wikipedia.org/wiki/War_(Edwin_Starr...,War,Edwin Starr,"War huh Yeah! Absolutely uh-huh, uh-huh huh Ye..."


In [14]:
# convert from pandas to spark dataframe
lyricsdf = sqlsc.createDataFrame(lyrics_pd_df)

In [15]:
# verify lyrics_S
lyricsdf.show(3)

+--------+----+--------------------+--------------------+-------------------+--------------------+
|position|year|          title.href|               title|             artist|              lyrics|
+--------+----+--------------------+--------------------+-------------------+--------------------+
|       1|1970|https://en.wikipe...|Bridge over Troub...|Simon and Garfunkel|When you're weary...|
|       2|1970|https://en.wikipe...|(They Long to Be)...|     The Carpenters|                   x|
|       3|1970|https://en.wikipe...|      American Woman|      The Guess Who|American woman, s...|
+--------+----+--------------------+--------------------+-------------------+--------------------+
only showing top 3 rows



In [16]:
# no longer need the pandas version, clear it out
del lyrics_pd_df

In [17]:
from pyspark.sql import functions as F
# add a column for better identity
#adapted from http://stackoverflow.com/questions/31450846/concatenate-columns-in-apache-spark-dataframe
lyricsdf = lyricsdf.withColumn('song_key',F.concat(F.col("year"), F.lit("-"), F.col("position")))
lyricsdf = lyricsdf.withColumn('year_str',F.concat(F.lit("_"),F.col("year")))

In [18]:
lyricsdf.show(3)

+--------+----+--------------------+--------------------+-------------------+--------------------+--------+--------+
|position|year|          title.href|               title|             artist|              lyrics|song_key|year_str|
+--------+----+--------------------+--------------------+-------------------+--------------------+--------+--------+
|       1|1970|https://en.wikipe...|Bridge over Troub...|Simon and Garfunkel|When you're weary...|  1970-1|   _1970|
|       2|1970|https://en.wikipe...|(They Long to Be)...|     The Carpenters|                   x|  1970-2|   _1970|
|       3|1970|https://en.wikipe...|      American Woman|      The Guess Who|American woman, s...|  1970-3|   _1970|
+--------+----+--------------------+--------------------+-------------------+--------------------+--------+--------+
only showing top 3 rows



In [19]:
#We cache the data to make sure it is only read once from disk
lyricsdf.cache()
print "How many songs do we have?", lyricsdf.count()
lyricsdf.printSchema()

How many songs do we have? 4500
root
 |-- position: long (nullable = true)
 |-- year: long (nullable = true)
 |-- title.href: string (nullable = true)
 |-- title: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- lyrics: string (nullable = true)
 |-- song_key: string (nullable = true)
 |-- year_str: string (nullable = true)



#Sample

Some initial sampling to take from each year.

In [20]:
PER_YEAR_SAMPLES=10

In [36]:
#(your code here)

def randomSubSampleLyrics(sparkdf,take=PER_YEAR_SAMPLES):    
    # generate spark pairs as a tuple
    br_pairs = sparkdf.map(lambda r: (r.year_str, r.song_key))
    
    # group by key for a list of reviews per business and collect
    br_grouped = br_pairs.groupByKey().mapValues(lambda x: list(x)).collect()
        
    #sample after collect
    br_sample = [np.random.choice(v, size=take, replace=False) for k,v in br_grouped]    
    
    #flatten into a list
    return list(itertools.chain.from_iterable(br_sample))
    
small_song_keys = randomSubSampleLyrics(lyricsdf)

In [37]:
small_song_keys[:5]

[u'2000-77', u'2000-90', u'2000-23', u'2000-82', u'2000-74']

#TODO: use sample to build smalldf

#NLP

In [ ]:
from pattern.en import parse
from pattern.en import pprint
from pattern.vector import stem, PORTER, LEMMA
punctuation = list('.,;:!?()[]{}`''\"@#$^&*+-|=~_')

In [ ]:
from sklearn.feature_extraction import text 
stopwords=text.ENGLISH_STOP_WORDS

In [ ]:
import re
regex1=re.compile(r"\.{2,}")
regex2=re.compile(r"\-{2,}")

###Quick Test

In [ ]:
parse("The world is the craziest place. I am working hard.", tokenize=True, lemmata=True)

In [ ]:
def get_parts(thetext):
    thetext=re.sub(regex1, ' ', thetext)
    thetext=re.sub(regex2, ' ', thetext)
    nouns=[]
    descriptives=[]
    for i,sentence in enumerate(parse(thetext, tokenize=True, lemmata=True).split()):
        nouns.append([])
        descriptives.append([])
        for token in sentence:
            #print token
            if len(token[4]) >0:
                if token[1] in ['JJ', 'JJR', 'JJS']:
                    if token[4] in stopwords or token[4][0] in punctuation or token[4][-1] in punctuation or len(token[4])==1:
                        continue
                    descriptives[i].append(token[4])
                elif token[1] in ['NN', 'NNS']:
                    if token[4] in stopwords or token[4][0] in punctuation or token[4][-1] in punctuation or len(token[4])==1:
                        continue
                    nouns[i].append(token[4])
    out=zip(nouns, descriptives)
    nouns2=[]
    descriptives2=[]
    for n,d in out:
        if len(n)!=0 and len(d)!=0:
            nouns2.append(n)
            descriptives2.append(d)
    return nouns2, descriptives2

### Quick Check

In [ ]:
get_parts("Have had many other items and just love the food. The patio...job was and...perfect. Lunch is good, and the only egg is great")

###Run Get Parts on Provided Data

In [ ]:
#(your code here)
review_parts = subdf.map(lambda r : get_parts(r.text))